In [9]:
from src.parameters import Params
from src.vocabulary import Vocabulary
from src.style_transfer import StyleTransfer
from src.greedy_decoding import Decoder
from src.generate_batches import preprocessSentences

In [2]:
params = Params()
vocab = Vocabulary()
vocab.loadVocabulary("data/yelp/vocabulary.pickle")
vocab.initializeEmbeddings(params.embedding_size)
model = StyleTransfer(params, vocab)

In [4]:
import torch
checkpoint = torch.load("data/models/yelp/model-2018-06-26-epoch_19-loss_68.143707")
model.load_state_dict(checkpoint)

In [24]:
import numpy as np

with open('data/yelp/dev/negative.txt', 'r') as fp:
    testSents = fp.readlines()[:16]
    
labels = np.array([0] * len(testSents))

In [5]:
decoder = Decoder(model, 20, 12, params)

In [27]:
encoder_inputs, generator_inputs, targets, lenghts = \
            model._sentencesToInputs(testSents)

In [33]:
model.eval_size = 16
model._computeHiddens(
                encoder_inputs, generator_inputs, labels, lenghts, True)
generatorOutputs, h_teacher = model._generateTokens(
            generator_inputs, model.originalHiddens, lenghts, True)

In [36]:
packedGenOutput = torch.nn.utils.rnn.pack_padded_sequence(
    generatorOutputs, lenghts, batch_first=True)[0]

model.rec_loss_criterion(
    packedGenOutput.view(-1, model.vocabulary.vocabSize),
    targets.view(-1))

tensor(3.9300, device='cuda:0')

In [55]:
sents = []
for index in range(16):
    tokensLogits = generatorOutputs[index, :, :]
    sent = []
    for j in range(16):
        logit = tokensLogits[j, :]
        sent.append(model.vocabulary.id2word[logit.argmax()])
    sents.append(" ".join(sent))
sents        

['it was obvious it was the same damn one he brought the first time . .',
 'major day nothing open , heard about this place figured it ok ok . . the',
 'the host that walked us to the table and left without to word hello . the',
 'the last couple years this place has been going smoothly hill to detailing the the the',
 'last night however it was way to thick and tasteless power ! the the the the',
 'i tried to eat it but it was disgusting appropriately ! the the the the the',
 "i pushed it by and did n't eat anymore than the the the the the the",
 'ok never going back to this place again to ! the the the the the the',
 'no sign of the manager ! ! the the the the the the the the the',
 'it smelled like rotten pretentious . . the the the the the the the the the',
 'i will never be back . . the the the the the the the the the',
 'it just gets worse . . the the the the the the the the the the',
 'the food tasted awful . . the the the the the the the the the the',
 'i am not exaggerating ma

In [48]:
testSents

['ok never going back to this place again .\n',
 'easter day nothing open , heard about this place figured it would ok .\n',
 'the host that walked us to the table and left without a word .\n',
 'it just gets worse .\n',
 'the food tasted awful .\n',
 'no sign of the manager .\n',
 'the last couple years this place has been going down hill .\n',
 'last night however it was way to thick and tasteless .\n',
 'it smelled like rotten urine .\n',
 'i am not exaggerating .\n',
 'this smelled bad !\n',
 'it was obvious it was the same damn one he brought the first time .\n',
 'i tried to eat it but it was disgusting .\n',
 'it tasted horrible !\n',
 "i pushed it aside and did n't eat anymore .\n",
 'i will never be back .\n']